In [4]:
#Start
from Research import functions
import time

In [2]:
functions.move_inv("right")

In [3]:
functions.shutdown()

In [ ]:
from scipy.spatial.transform import Rotation as R
import numpy as np

print(np.around(R.from_euler('y', np.deg2rad(50)).as_matrix(), 3))

In [1]:
from reachy_sdk import ReachySDK
from reachy_sdk.trajectory import goto
from reachy_sdk.trajectory.interpolation import InterpolationMode

reachy = ReachySDK(host='127.0.0.1')

In [2]:
import time
time.sleep(7)
reachy.l_arm.forward_kinematics()

array([[-0.93041691, -0.07393605, -0.35896772,  0.30971189],
       [-0.35692519, -0.03965655,  0.93329083,  0.27420855],
       [-0.08323926,  0.99647419,  0.01050748, -0.1504515 ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [ ]:
import numpy as np
A = np.array([
    [-0.27439431, -0.1039175 , -0.95598584,  0.43870858],
       [-0.96111198,  0.06186045,  0.2691413 ,  0.17362613],
       [ 0.03116922,  0.99266028, -0.11685051, -0.25943446],
       [ 0.        ,  0.        ,  0.        ,  1.        ]
])

joint_pos_A = reachy.l_arm.inverse_kinematics(A)

reachy.turn_on('l_arm')
goto({joint: pos for joint,pos in zip(reachy.l_arm.joints.values(), joint_pos_A)}, duration=1.0)
time.sleep(1)
reachy.turn_off_smoothly('l_arm')

In [ ]:
import time
# put the joints in stiff mode
reachy.turn_on('r_arm')
while 1:
    # use the goto function
    goto({joint: pos for joint,pos in zip(reachy.r_arm.joints.values(), joint_pos_A)}, duration=1.0)
    time.sleep(0.5)
    goto({joint: pos for joint,pos in zip(reachy.r_arm.joints.values(), joint_pos_B)}, duration=1.0)
    time.sleep(0.5)
    goto({joint: pos for joint,pos in zip(reachy.r_arm.joints.values(), joint_pos_C)}, duration=1.0)
    time.sleep(0.5)
    goto({joint: pos for joint,pos in zip(reachy.r_arm.joints.values(), joint_pos_D)}, duration=1.0)

# put the joints back to compliant mode
# use turn_off_smoothly to prevent the arm from falling hard
reachy.turn_off_smoothly('r_arm')

In [ ]:
#Turn off
def shutdown():
    reachy.turn_off('head')
    reachy.turn_off_smoothly('r_arm')
    reachy.turn_off_smoothly('l_arm')    

In [ ]:
shutdown()

In [ ]:
#VARIABLES
r_gripper = 0
l_gripper = 0

l_shoulder_p = 0
l_shoulder_r = 30
l_arm_y = -40
l_elbow_p = -90
l_forearm_y = -90
l_wrist_p = -20    
l_wrist_r = 40   

r_shoulder_p = 0
r_shoulder_r = -30
r_arm_y = 40
r_elbow_p = -90
r_forearm_y = 90
r_wrist_p = -20    
r_wrist_r = -40    

In [ ]:
#rest left arm
def rest_l():
    reachy.turn_off_smoothly('l_arm') 

In [ ]:
#rest right arm
def rest_r():
    reachy.turn_off_smoothly('r_arm') 

In [ ]:
#Move left arm
def move_l_arm(pos):
    reachy.turn_on('l_arm')
    move(pos)

In [ ]:
#Move right arm
def move_r_arm(pos):
    reachy.turn_on('r_arm')
    move(pos)

In [ ]:
def move(pos):
    goto(
        goal_positions=pos,
        duration=1.0,
        interpolation_mode=InterpolationMode.MINIMUM_JERK
        )

In [ ]:
#move gripper
def move_l_hand(value):
    if(value=='open'): value = 20
    if(value=='close'): value = -20
    l_gripper = value
    reachy.turn_on('l_arm')
    gripper = {
        reachy.l_arm.l_gripper: l_gripper #(-20 --> CLOSE / 20 --> OPEN)
    }
    move(gripper)

In [ ]:
#move gripper
def move_r_hand(value):
    if(value=='open'): value = 20
    if(value=='close'): value = -20
    r_gripper = value
    print(r_gripper)
    reachy.turn_on('r_arm')
    gripper = {
        reachy.r_arm.r_gripper: r_gripper #(-20 --> CLOSE / 20 --> OPEN)
    }
    goto(
        goal_positions=gripper,
        duration=1.0,
        interpolation_mode=InterpolationMode.MINIMUM_JERK
        )

In [ ]:
#Hand + arm to desired pos for project
def adjust_l_pos():
    left_angled_position = {
    reachy.l_arm.l_shoulder_pitch: l_shoulder_p,
    reachy.l_arm.l_shoulder_roll: l_shoulder_r,
    reachy.l_arm.l_arm_yaw: l_arm_y,
    reachy.l_arm.l_elbow_pitch: l_elbow_p,
    reachy.l_arm.l_forearm_yaw: l_forearm_y,
    reachy.l_arm.l_wrist_pitch: l_wrist_p,
    reachy.l_arm.l_wrist_roll: l_wrist_r,
    }
    move_l_hand(l_gripper)
    move_l_arm(left_angled_position)

In [ ]:
adjust_l_pos()

In [ ]:
reachy.r_arm.forward_kinematics()

In [ ]:
def adjust_r_pos():
    right_angled_position = {
        reachy.r_arm.r_shoulder_pitch: r_shoulder_p,
        reachy.r_arm.r_shoulder_roll: r_shoulder_r,
        reachy.r_arm.r_arm_yaw: r_arm_y,
        reachy.r_arm.r_elbow_pitch: r_elbow_p,
        reachy.r_arm.r_forearm_yaw: r_forearm_y,
        reachy.r_arm.r_wrist_pitch: r_wrist_p,
        reachy.r_arm.r_wrist_roll: r_wrist_r,
    }
    #print(r_gripper)
    move_r_hand(r_gripper)
    move_r_arm(right_angled_position)

In [ ]:
adjust_r_pos()

In [ ]:
time.sleep(3)

In [ ]:
adjust_l_pos()
adjust_r_pos()

posit = [20,-20,20,-20,20,-20]
gripper_pos = [0,-20,0,-20,0,-20]
elbow_pos = [-90,-45,-90,-45,-90, -45]
wrist_pos = [20,-20,20,-20,20,-20]
i=0
for pos in posit:
    r_gripper = -gripper_pos[i]
    r_arm_y+=pos
    r_elbow_p = elbow_pos[i]
    r_wrist_p = -wrist_pos[i]
    adjust_r_pos()
    time.sleep(0.5)
    i+=1
time.sleep(1)
rest_r()
rest_l()

In [ ]:
#threading?
#def twitch():
    #move arms instead of holding them still

In [ ]:
#rest arms
rest_r()
rest_l()

In [ ]:
import face_recognition as fr
import cv2

def get_player_amount():
  video_capture = reachy.left_camera
  frame = video_capture.last_frame
  rgb_frame = frame[:, :, ::-1]
  face_locations = fr.face_locations(rgb_frame)
  return(len(face_locations))

In [ ]:
variable = get_player_amount()
print(variable)

In [ ]:
video_capture = cv2.VideoCapture()
print(video_capture)

In [ ]:
reachy_audio = reachyAudio.ReachyAudio()

In [ ]:
reachy_audio.speak("see you later aligator")

In [ ]:
def speak(text):
    tts = gTTS(text)
    tts.save('voiceToAlter.mp3')
    sound = AudioSegment.from_mp3('voiceToAlter.mp3')
    sound.export('voiceToAlter.wav', format='wav')

    # Alter the previously created audio file
    #outputFileName = self.diodeRingModulator('voiceToAlter.wav')

    # Play the altered audio file
    reachyAudioPlayerRecorderObject.playAudio('voiceToAlter.wav')
    time.sleep(0.5)

In [ ]:
def playAudio(self, wavFileName):
        """Play a WAV file.

        :param wavFileName: Name of the WAV file to play.
        """
        try:
            # Open the wav file
            wf = wave.open(wavFileName, 'rb')

            # Create the PyAudio object and open the PyAudio stream
            p = pyaudio.PyAudio()
            stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                            channels=wf.getnchannels(),
                            rate=wf.getframerate(),
                            output=True)

            # Read the wav file until his end
            data = wf.readframes(self.chunk)

            while data != b'':
                stream.write(data)
                data = wf.readframes(self.chunk)

            # Terminate and close all the processes
            stream.stop_stream()
            stream.close()

            p.terminate()

            wf.close()

        except Exception as e:
            print("Exception: " + str(e))


In [ ]:
said = reachy_audio.recognizeSpeech()
reachy_audio.speak("Hello there, my name is reachy.")

In [ ]:
def adjust_head():
    head_position = {
        reachy.head.joints.l_antenna: -0.50,
        reachy.head.joints.r_antenna: -1.50,
        reachy.head.joints.neck_disk_top: -54.72,
        reachy.head.joints.neck_disk_middle: -69.77,
        reachy.head.joints.neck_disk_bottom: -54.11,
    }
    move(head_position)

In [ ]:
reachy.turn_on('head')
adjust_head()

In [ ]:
antenna_positions = [-100, -130, -100, -130, -100, -130]

In [ ]:
reachy.turn_on('head')
for pos in antenna_positions:
    reachy.head.l_antenna.goal_position = pos
    time.sleep(0.5)

In [ ]:
shutdown()